# アンサンブル学習

#### 小さなデータセットの用意
>以前も利用した回帰のデータセットを用意します。
>
>House Prices: Advanced Regression Techniques
>
>この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。
>
>train.csvを学習用（train）8割、検証用（val）2割に分割してください。
>
#### scikit-learn
>単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

## 【問題1】ブレンディングのスクラッチ実装  
>ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。



### ブレンディングとは
>ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。
>
>- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
>- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
>- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）
>
>重要なのはそれぞれのモデルが大きく異なることです。
>
>回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。
>
>《補足》
>
>分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# データセットcsvをpandasに読み込む

csv_path = "./Kaggle_data/train.csv" # ファイル名（パス）を指定する
df_data = pd.read_csv(csv_path)

# 条件に従って抜き出し
df_X = df_data[['GrLivArea', 'YearBuilt']]
df_y = df_data['SalePrice']

df = pd.concat([df_X, df_y], axis=1)

display(df)

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000
...,...,...,...
1455,1647,1999,175000
1456,2073,1978,210000
1457,2340,1941,266500
1458,1078,1950,142125


In [3]:
# 特徴量（説明変数）をX、正解（目的変数）をyというndarrayに格納

X = np.array(df[['GrLivArea','YearBuilt']])
y = np.array(df['SalePrice'])
print(X)
print(y)

[[1710 2003]
 [1262 1976]
 [1786 2001]
 ...
 [2340 1941]
 [1078 1950]
 [1256 1965]]
[208500 181500 223500 ... 266500 142125 147500]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1168, 2)
(1168,)
(292, 2)
(292,)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error

In [7]:
# 精度の一覧表用の空のリストを用意
verification_result = []


#最近傍法
verification_neigh = ['k-nearest neighbors']

# 近傍点を５に設定して学習
neigh_baseline = KNeighborsClassifier(n_neighbors=5)# インスタンス化
neigh_baseline.fit(X_train_scaled, y_train)# 学習
y_pred_n = neigh_baseline.predict(X_test_scaled)# 予測

# 平均二乗誤差（Mean Squared Error, MSE）    
verification_neigh.append(mean_squared_error(y_test,  y_pred_n))


# 線形回帰
verification_linear = ['LinearRegression']

linear_baseline = SGDClassifier()
linear_baseline.fit(X_train_scaled, y_train)
y_pred_li = linear_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）    
verification_linear.append(mean_squared_error(y_test,  y_pred_li))



# SVM
verification_svm = ['SVM']

svm_baseline = SVC()
svm_baseline.fit(X_train_scaled, y_train)
y_pred_s = svm_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）
verification_svm.append(mean_squared_error(y_test, y_pred_s))



# 決定木
verification_tree = ['DecisionTree']

tree_baseline = DecisionTreeClassifier()
tree_baseline.fit(X_train_scaled, y_train)
y_pred_t = tree_baseline.predict(X_test_scaled)

# 平均二乗誤差（Mean Squared Error, MSE）
verification_tree.append(mean_squared_error(y_test, y_pred_t))



# 表を作成するために計算結果を２次元配列にする
verification_result = [
            verification_neigh,
            verification_linear, 
            verification_svm, 
            verification_tree, 
        ]


# 行と列のインデックスようのリストを用意
data_columns=['Model', '平均二乗誤差（MSE）']

# pandas のデータフレームにする
df_verification = pd.DataFrame(data=verification_result, columns=data_columns)

print("今回のベースライン")
display(df_verification)


今回のベースライン


,Model,平均二乗誤差（MSE）
0,k-nearest neighbors,4.256201e+09
1,LinearRegression,5.991151e+09
2,SVM,2.673653e+09
3,DecisionTree,3.551295e+09


SVM、決定木、最近傍法、線形回帰の順に精度が高い。

In [8]:
class ScratchEnsembleLearning():
    """
    アンサンブル学習

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, model_instance, weight):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        INSTANCE_0 = model_instance[0]
        INSTANCE_1 = model_instance[1]
        INSTANCE_2 = model_instance[2]

        INSTANCE_0.fit(X, y)
        INSTANCE_1.fit(X, y)
        INSTANCE_2.fit(X, y)
        
        self.instance_0 = INSTANCE_0
        self.instance_1 = INSTANCE_1
        self.instance_2 = INSTANCE_2
        
        
    def predict(self, X):
        """
        推定する
        """
        pred_0 = self.instance_0.predict(X_test)
        pred_1 = self.instance_1.predict(X_test)
        pred_2 = self.instance_2.predict(X_test)
        
        pred = (pred_0*weight[0] + pred_1*weight[1] + pred_2*weight[2])#/3
        
        self.pred = pred
        self.pred_0 = pred_0
        self.pred_1 = pred_1
        self.pred_2 = pred_2
        
        return pred
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        # 精度の一覧表用の空のリストを用意
        verification_result = []
        MSE_0 = mean_squared_error(y_test, self.pred_0)
        MSE_1 = mean_squared_error(y_test, self.pred_1)
        MSE_2 = mean_squared_error(y_test, self.pred_2)
        
        # 表を作成するために計算結果を２次元配列にする
        verification_MSE_0 = ['INSTANCE_0', MSE_0]
        verification_MSE_1 = ['INSTANCE_1', MSE_1]
        verification_MSE_2 = ['INSTANCE_3', MSE_2]
        verification_MSE = ['EnsembleLearning', MSE]
        verification_result = [
                    verification_MSE_0, 
                    verification_MSE_1, 
                    verification_MSE_2, 
                    verification_MSE
                ]
        # pandas のデータフレームにする
        df_verification = pd.DataFrame(data=verification_result, columns=data_columns)
        display(df_verification)
        
        return MSE
        

In [9]:
# インスタンス化
neigh = KNeighborsClassifier(n_neighbors=5)
linear = SGDClassifier()
svm = SVC()
tree = DecisionTreeClassifier()

#### パターン１

In [10]:
# 引数
model_instance = [linear, svm, tree]
weight = [0.3, 0.4, 0.3]

# インスタンス化
ensemble_learning_1 = ScratchEnsembleLearning()
ensemble_learning_1.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_1.predict(X_test_scaled)
ensemble_learning_1.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,4.038873e+10
1,INSTANCE_1,8.617359e+09
2,INSTANCE_3,9.909679e+10
3,EnsembleLearning,2.354077e+10


23540767014.166298

#### パターン２  
SVMがさらに強く出るように比率を変更。また、決定木は誤差が大きいのであまり効かないよう重みを下げた。

In [11]:
# 引数
model_instance = [linear, svm, tree]
weight = [0.2, 0.7, 0.1]

# インスタンス化
ensemble_learning_2 = ScratchEnsembleLearning()
ensemble_learning_2.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_2.predict(X_test_scaled)
ensemble_learning_2.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,9.605495e+09
1,INSTANCE_1,8.617359e+09
2,INSTANCE_3,9.909679e+10
3,EnsembleLearning,7.305266e+09


7305266206.890411

#### パターン３  
線形回帰をより精度が高い最近傍法に変更。

In [12]:
# 引数
model_instance = [neigh, svm, tree]
weight = [0.2, 0.7, 0.1]

# インスタンス化
ensemble_learning_3 = ScratchEnsembleLearning()
ensemble_learning_3.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_3.predict(X_test_scaled)
ensemble_learning_3.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,6.917272e+09
1,INSTANCE_1,8.617359e+09
2,INSTANCE_3,9.909679e+10
3,EnsembleLearning,6.910175e+09


6910174768.534246

#### パターン４  
最近傍法が意外と精度が高いので比率を上げてみる。

In [13]:
# 引数
model_instance = [neigh, svm, tree]
weight = [0.3, 0.4, 0.1]

# インスタンス化
ensemble_learning_4 = ScratchEnsembleLearning()
ensemble_learning_4.fit(X_train_scaled, y_train, model_instance, weight)
y_pred = ensemble_learning_4.predict(X_test_scaled)
ensemble_learning_4.MSE(y_test, y_pred)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,6.917272e+09
1,INSTANCE_1,8.617359e+09
2,INSTANCE_3,9.909679e+10
3,EnsembleLearning,7.365754e+09


7365754440.109589

### 結論  
最近傍法, SVM, 決定木を0.2, 0.7, 0.1の重みづけで出力すると一番誤差が少ないことがわかった。  
個別で精度が高くいものを増やしたり、比率を上げたりしてみたが、組み合わせ等の問題で  
全体では精度が上がらないということがあることがわかった。

## 【問題2】バギングのスクラッチ実装  
>バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。



## バギングとは  
>バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ **ブートストラップサンプル** ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。
>
>sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation
>
>scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。
>
>推定結果の平均をとる部分はブースティングと同様の実装になります。

In [14]:
import copy

class ScratchBagging():
    """
    バギング

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, model_instance, test_size=0.2):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        X_train_0, _ , y_train_0, _ = train_test_split(X, y, test_size=test_size, shuffle=True)
        X_train_1, _ , y_train_1, _ = train_test_split(X, y, test_size=test_size, shuffle=True)  
        X_train_2, _ , y_train_2, _ = train_test_split(X, y, test_size=test_size, shuffle=True)
        
        model_0 = model_instance[0]
        model_1 = model_instance[1]
        model_2 = model_instance[2]
        
        model_0.fit(X_train_0, y_train_0)
        model_1.fit(X_train_1, y_train_1)
        model_2.fit(X_train_2, y_train_2)
        
        self.instance_0 = model_0
        self.instance_1 = model_1
        self.instance_2 = model_2
        
        
    def predict(self, X):
        """
        推定する
        """
        pred_tmp_0 = self.instance_0.predict(X)
        pred_tmp_1 = self.instance_1.predict(X)
        pred_tmp_2 = self.instance_2.predict(X)
        
        pred = (pred_tmp_0+pred_tmp_1+pred_tmp_2)/3
        
        self.pred_0 = pred_tmp_0
        self.pred_1 = pred_tmp_1
        self.pred_2 = pred_tmp_2
        
        return pred
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        # 精度の一覧表用の空のリストを用意
        verification_result = []
        MSE_0 = mean_squared_error(y_test, self.pred_0)
        MSE_1 = mean_squared_error(y_test, self.pred_1)
        MSE_2 = mean_squared_error(y_test, self.pred_2)
        
        # 表を作成するために計算結果を２次元配列にする
        verification_MSE_0 = ['INSTANCE_0', MSE_0]
        verification_MSE_1 = ['INSTANCE_1', MSE_1]
        verification_MSE_2 = ['INSTANCE_3', MSE_2]
        verification_MSE = ['Bagging', MSE]
        verification_result = [
                    verification_MSE_0, 
                    verification_MSE_1, 
                    verification_MSE_2, 
                    verification_MSE
                ]
        # pandas のデータフレームにする
        df_verification = pd.DataFrame(data=verification_result, columns=data_columns)
        display(df_verification)
        
        return MSE
        

#### パターン1
ベースラインの結果が良かったSVMで実施。

In [15]:
# 引数
svm_bagging_0 = SVC()
svm_bagging_1 = SVC()
svm_bagging_2 = SVC()

model_instance = [svm_bagging_0, svm_bagging_1, svm_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,3.665229e+09
1,INSTANCE_1,2.893126e+09
2,INSTANCE_3,3.082564e+09
3,Bagging,2.822355e+09


2822355470.094368

#### パターン２
ベースラインがあまり良くなかった決定木で実施。

In [16]:
# 引数
tree_bagging_0 = DecisionTreeClassifier()
tree_bagging_1 = DecisionTreeClassifier()
tree_bagging_2 = DecisionTreeClassifier()

model_instance = [tree_bagging_0, tree_bagging_1, tree_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,2.824472e+09
1,INSTANCE_1,3.898981e+09
2,INSTANCE_3,3.830686e+09
3,Bagging,2.612077e+09


2612077237.097412

#### パターン３
最近傍法で実施。

In [17]:
# 引数
neigh_bagging_0 = KNeighborsClassifier(n_neighbors=5)
neigh_bagging_1 = KNeighborsClassifier(n_neighbors=5)
neigh_bagging_2 = KNeighborsClassifier(n_neighbors=5)

model_instance = [neigh_bagging_0, neigh_bagging_1, neigh_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,4.430251e+09
1,INSTANCE_1,4.291967e+09
2,INSTANCE_3,3.965840e+09
3,Bagging,4.054436e+09


4054435800.6023583

#### パターン４
線形回帰で実施。

In [18]:
# 引数
linear_bagging_0 = SGDClassifier()
linear_bagging_1 = SGDClassifier()
linear_bagging_2 = SGDClassifier()

model_instance = [linear_bagging_0, linear_bagging_1, linear_bagging_2]

# インスタンス化
bagging_1 = ScratchBagging()
bagging_1.fit(X_train_scaled, y_train, model_instance, test_size=0.2)
y_pred_b = bagging_1.predict(X_test_scaled)
bagging_1.MSE(y_test, y_pred_b)

,Model,平均二乗誤差（MSE）
0,INSTANCE_0,5.579812e+09
1,INSTANCE_1,6.154325e+09
2,INSTANCE_3,6.361193e+09
3,Bagging,5.159755e+09


5159755172.681888

### 結論  
最近傍法, 決定木はバギングによって精度が向上した。

## 【問題3】スタッキングのスクラッチ実装  
>スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。



In [19]:
from sklearn.model_selection import KFold

In [20]:
class ScratchStacking():
    """
    スタッキング

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
      
    Attributes
    ----------
    self.pred_list
    　　
    self.instance_list
    　　
    self.instance_l2
      
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
        
        
    def fit(self, X, y, INSTANCE, INSTANCE_L2):
        """
        学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        ############ ステージ０　############
        # 推定保管用
        pred_list = []
        
        # モデル数分ループ
        for i in range(3):
            # データを４分割
            kf = KFold(n_splits=4)
            # データ分割を実施
            kf.get_n_splits(X)
            
            # 推定保管用
            pred_tmp = []
            
            # ループ回数のカウント用
            j = 0
            #for j, (train_index, test_index) in enumerate(zip(kf.split(X))):
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # 学習
                INSTANCE[i][j].fit(X_train, y_train)
                # 推定
                y_pred = INSTANCE[i][j].predict(X_test)
                pred_tmp.extend(y_pred)
                
                j += 1
            
            # OOF
            pred_list.append(pred_tmp)

        # OOF の保管
        self.pred_list = np.array(pred_list).T
        # インスタンスの保管
        self.instance_list = np.array(INSTANCE)
        
        ############ ステージN　############
        # pred を 学習用に分割
        X_train_l2, X_test_l2, y_train_l2, y_test_l2 = train_test_split(self.pred_list, y, test_size=0.2, random_state=0)

        # 学習
        INSTANCE_L2.fit(X_train_l2, y_train_l2)
        # インスタンスの保管
        self.instance_l2 = INSTANCE_L2
        
        
    def predict(self, X):
        """
        推定する
        """
        ############ ステージ０　############
        # 推定保管用
        pred_list = []
        for i in range(3):
            # 推定保管用
            pred_tmp = []
            for j in range(4):
                # 推定
                y_pred = self.instance_list[i, j].predict(X)
                pred_tmp.append(y_pred)
            # 同一モデルないで平均を取る
            pred_tmp = np.mean(np.array(pred_tmp).T, axis=1)
            pred_list.append(pred_tmp)
            
        # 推定データの作成
        pred_list = np.array(pred_list).T
        
        ############ ステージN　############
        y_pred_l2 = self.instance_l2.predict(pred_list)
        print(y_pred_l2)
        
        return y_pred_l2
    
    
    def MSE(self, y_test, y_pred):
        """
        平均二乗誤差（Mean Squared Error, MSE）
        """
        MSE = mean_squared_error(y_test, y_pred)
        
        return MSE
        

#### パターン1
ベースラインとして単独で実施したときに最も精度の良かったSVCのみで構成して実施。

In [21]:
# 引数

# L１のインスタンス
svm_stacking_a0 = SVC()
svm_stacking_a1 = SVC()
svm_stacking_a2 = SVC()
svm_stacking_a3 = SVC()
instance_a = [svm_stacking_a0, svm_stacking_a1, svm_stacking_a2, svm_stacking_a3]


svm_stacking_b0 = SVC()
svm_stacking_b1 = SVC()
svm_stacking_b2 = SVC()
svm_stacking_b3 = SVC()
instance_b = [svm_stacking_b0, svm_stacking_b1, svm_stacking_b2, svm_stacking_b3]


svm_stacking_c0 = SVC()
svm_stacking_c1 = SVC()
svm_stacking_c2 = SVC()
svm_stacking_c3 = SVC()
instance_c = [svm_stacking_c0, svm_stacking_c1, svm_stacking_c2, svm_stacking_c3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = SVC()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[190000 140000 140000 190000 140000 119000 174000 140000 190000 140000
 174000 180000 190000 140000 140000 140000 190000 140000 140000 140000
 140000 140000 135000 174000 190000 140000 174000 135000 190000 140000
 140000 190000 140000 130000 190000 174000 174000 140000 190000 340000
 140000 140000 180000 190000 340000 140000 115000 140000 190000 135000
 340000 140000 140000 135000 174000 140000 140000 190000 140000 115000
 140000 140000 140000 140000 174000 140000 135000 190000 115000 190000
 174000 140000 115000 174000 140000 190000 140000 119000 340000 140000
 140000 140000 140000 140000 190000 174000 140000 174000 180000 140000
 190000 180000 174000 190000 180000 140000 174000 140000 140000 140000
 190000 190000 140000 140000 119000 340000 174000 119000 174000 140000
 119000 140000 190000 140000 174000 180000 340000 140000 190000 190000
 140000 174000 140000 180000 174000 174000 190000 180000 180000 180000
 174000 180000 190000 180000 140000 190000 140000 174000 115000 174000
 14000

4035473216.479452

単独で実施した時が2827616000のため、むしろ精度が下がった。  
単独のモデルを使うより、いろいろなモデルを組み合わせる方が良いと思われる。

#### パターン２
ステージ０をSVM、最近傍法、決定木で構成。ステージNは線形回帰で構成。

In [22]:
# 引数

# L１のインスタンス
svm_stacking_0 = SVC()
svm_stacking_1 = SVC()
svm_stacking_2 = SVC()
svm_stacking_3 = SVC()
instance_a = [svm_stacking_0, svm_stacking_1, svm_stacking_2, svm_stacking_3]

neigh_stacking_0 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_1 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_2 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_3 = KNeighborsClassifier(n_neighbors=5)
instance_b = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

tree_stacking_0 = DecisionTreeClassifier()
tree_stacking_1 = DecisionTreeClassifier()
tree_stacking_2 = DecisionTreeClassifier()
tree_stacking_3 = DecisionTreeClassifier()
instance_c = [tree_stacking_0, tree_stacking_1, tree_stacking_2, tree_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = SGDClassifier()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[189000 189000 189000 189000 153500 189000 189000 189000 153500 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 153500 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 168000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 153500 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 168000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 153500 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 189000 189000 189000 189000 189000 189000 189000 189000 189000 153500
 189000 189000 189000 189000 189000 189000 189000 189000 189000 189000
 18900

7790887136.739726

精度が悪化した。ステージNは単独モデルによる学習になるため、精度の高いモデルの方が良いのではないか。

#### パターン３  
最終ステージは精度の高いSVMで構成。

In [25]:
# 引数

# L１のインスタンス
neigh_stacking_0 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_1 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_2 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_3 = KNeighborsClassifier(n_neighbors=5)
instance_a = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

tree_stacking_0 = DecisionTreeClassifier()
tree_stacking_1 = DecisionTreeClassifier()
tree_stacking_2 = DecisionTreeClassifier()
tree_stacking_3 = DecisionTreeClassifier()
instance_b = [tree_stacking_0, tree_stacking_1, tree_stacking_2, tree_stacking_3]

linear_stacking_0 = SGDClassifier()
linear_stacking_1 = SGDClassifier()
linear_stacking_2 = SGDClassifier()
linear_stacking_3 = SGDClassifier()
instance_c = [linear_stacking_0, linear_stacking_1, linear_stacking_2, linear_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = SVC()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[180000 140000 140000 140000 135000 110000 180000 140000 325000 140000
 180000 140000 180000 140000 140000 140000 180000 140000 140000 140000
 140000 140000 135000 155000 155000 135000 180000 110000 290000 140000
 140000 180000 140000 180000 260000 155000 180000 140000 180000 290000
 180000 135000 155000 290000 290000 140000 135000 140000 140000 135000
 325000 140000 140000 110000 180000 140000 140000 173000 140000 110000
 140000 140000 140000 140000 180000 140000 140000 290000 135000 180000
 180000 135000 135000 155000 140000 290000 140000 110000 290000 140000
 140000 140000 140000 140000 180000 140000 140000 180000 180000 140000
 180000 140000 180000 180000 155000 155000 180000 140000 140000 140000
 180000 290000 140000 140000 110000 290000 155000 110000 155000 140000
  86000 140000 180000 135000 140000 140000 290000 140000 180000 180000
 140000 180000 140000 155000 155000 174000 290000 155000 140000 140000
 155000 140000 180000 140000 140000 180000 140000 155000 135000 155000
 14000

3357875175.3835616

精度が下がったことを確認。

#### パターン４  
最終ステージを決定木でも実施。

In [26]:
# 引数

# L１のインスタンス
svm_stacking_0 = SVC()
svm_stacking_1 = SVC()
svm_stacking_2 = SVC()
svm_stacking_3 = SVC()
instance_a = [svm_stacking_0, svm_stacking_1, svm_stacking_2, svm_stacking_3]

neigh_stacking_0 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_1 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_2 = KNeighborsClassifier(n_neighbors=5)
neigh_stacking_3 = KNeighborsClassifier(n_neighbors=5)
instance_b = [neigh_stacking_0, neigh_stacking_1, neigh_stacking_2, neigh_stacking_3]

linear_stacking_0 = SGDClassifier()
linear_stacking_1 = SGDClassifier()
linear_stacking_2 = SGDClassifier()
linear_stacking_3 = SGDClassifier()
instance_c = [linear_stacking_0, linear_stacking_1, linear_stacking_2, linear_stacking_3]

model_instance = [instance_a, instance_b, instance_c]

# L2のインスタンス
l2__instance = DecisionTreeClassifier()



# インスタンス化
stacking_1 = ScratchStacking()
stacking_1.fit(X_train_scaled, y_train, model_instance, l2__instance)

# 推定
y_pred_b = stacking_1.predict(X_test_scaled)
stacking_1.MSE(y_test, y_pred_b)

[135000 124500 109900 187000  83500 147000 176485 124500 184750 139600
 224000 162900 233230 117500 109900 181000 239000 130000 164900 179900
 124500 140200 180000 160000 125000 128500 174000 180000 225000  80500
 165000 226000 128900 169000 219210 179400 168500 235000 228000 281213
 174000 161000 159895 219500 281213 107500 102000 129500 162900  81000
 380000 147000 143750  34900 162000 117500 134000 285000 136500 110500
 115000 145000 181000 107000 187000 165000 109900 211000 180000 197900
 171750 110500 109900 176485 109900 225000 102000 125500 276000  99500
 128900 124500 117500 130000 239000 164700  87000 174000 178000 121600
 259000 217500 179200 233230 176485 181000 174000 179900 117500 164900
 197900 233230  99500 147400  86000 402861 221500 129000 174000 117500
  86000 115000 179200 110500 180000 118964 333168 115000 184000 255000
 139600 176485 102000 147400 172500 224000 255000 164500 217500 120000
 160000 217500 204750 147400 117500 259000 140000 160000 109900 162000
 10990

3945142655.020548

それなりにいい結果が出た。

#### 考察  
個別で精度が高いからと言って単独のモデルでスタッキングを構成しても精度は上がらず、 
むしろ下がる結果になった。  
また、いろいろなモデルを組み合わせるとしても、最終ステージのモデルはできるだけ精度の高い  
ものを使用する方が良いことがわかった。

以上